In [5]:
cc.VerificationHandler.close_browser()

In [4]:
VerificationHandler.close_browser()

In [12]:
import numpy as np
import pandas as pd

In [1]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [2]:
% run contactChecker.py

Local Contact Checker Ready


In [5]:
ContactSheetOutput.set_output(contactKeys)
VerificationHandler.set_orgRecords(dm.OrgSession(orgRecords))
VerificationHandler.set_contactRecords(cr)
print('Local Contact Checker Ready')

RECORD OUTPUT READY
Local Contact Checker Ready


In [6]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

In [7]:
org  = orgsForToday[0]

vh = MotherSetVerifier(org)

In [245]:
org  = orgsForToday[0]

vh = ContactScraperVerifier(org)

In [246]:
VerificationHandler.orgRecords.orgSessionStatusCheck()

['Sun Oct 22, 2017  02:28:32', '2.954', 's', 'Good']

In [9]:
ContactSheetOutput.currentRow

243

In [10]:
vh.write_contact_pointers()

In [248]:
vh.pointers

In [254]:
vh.extenders[2].tom_rocket(nathanStarts[7])

'Administrative Assistant'

In [255]:
vh.extenders[2].nathan_rocket(tomStarts[7])

"Davana O'Brien"

In [244]:
class ContactScraperVerifier(MotherSetVerifier):
    def __init__(self, org):
        MotherSetVerifier.__init__(self, org)
        self.pointers = self.verifiedPointers
        
        # Get Grand Mother elements
        self.gmElements, self.grandMotherMatrix = ContactScraperVerifier.getGrandMotherElements(self.pointers)
        
        self.noGm = len(self.gmElements)
        
        if self.noGm == 1: ## Single Grandmother Case
            self.gm = self.gmElements[0]
            
            if distinct_gm(self.gm, self.pointers): ## Grandmother is distinct
            ## Extender Model Selection
                self.extenders = [Extender(self.gm, vp) for vp in self.pointers]
            
            else:  ## GrandMother is not distinct
                ## Extender Model Selection
                self.extenders = [RocketOnlyExtender(self.gm, vp) for vp in self.pointers]
            
        else:
            ## Extender Model Selection for Multiple GrandMothers
            self.extenders = None
            
    @staticmethod
    def getGrandMotherElements(pointers):
        ## Identify Grandmother elements
        gmElements = []
        gmMatrix = []

        for i in range(len(pointers)):
            igmElements = []
            for j in range(i):
                ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
                if pointers[i].get_mother_element() is pointers[j].get_mother_element():
                    gm = pointers[i].get_mother_element()
                else:
                    gm = pointers[i].common_parent(pointers[j])
                # Append Match to Grand Mother Matrix
                igmElements.append(gm)

                # Check to see if this is a new grand mother element,
                # if so append to the gmElements list of unique grandmother elements 
                if gm not in gmElements:
                    gmElements.append(gm)

            # Append Matrix Row
            gmMatrix.append(igmElements)

        grandMotherMatrix = np.matrix(gmMatrix)
        return (gmElements, grandMotherMatrix)
    
    @staticmethod
    def distinct_gm(gm, pts):
        if len(pts) == 0:
            return True
        if gm is pts[0].get_mother_element():
            return False
        else:
            return distinct_gm(gm, pts[1:])

In [242]:
class Extender(object):
    def __init__(self, gm, pointer):
        self.gm = gm
        self.vp = pointer
        
        self.tag_nathans()
        self.tag_toms()
        
        self.tom_missile = self.mother_to_tom_missile()
        self.tom_rocket = self.nathan_to_tom_rocket()
        
        self.nathan_missile = self.mother_to_nathan_missile()
        self.nathan_rocket = self.tom_to_nathan_rocket()
        
        
        
    ## Tagging Functions ------------------------------------ 
    ##
    def tag_nathans(self):
        return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.nathan.parent, 'nathan', 0)

    def tag_toms(self):
        return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.tom.parent, 'tom', 0)
    
    def reset_tree(self):
        ## Clear all tom, nathan and sib atributes
        for tomTag in self.gm.find_all(has_tom):
            del tomTag['tom']    
        for nathanTag in self.gm.find_all(has_nathan):
            del nathanTag['nathan'] 
    
    @staticmethod
    def parent_cycle_up(motherElement, element, atr, num):
        ## for the bertha case we stop at the element that is right befor the mother elemenet
        if element.parent is motherElement:
            element[atr] = num
            element['sib'] = atr
        else:
            element[atr] = num
            return Extender.parent_cycle_up(motherElement, element.parent, atr, num + 1)
    
    @staticmethod
    def contents_position(elm):
        return Extender.contents_position_loop(elm, 0)

    @staticmethod
    def contents_position_loop(elm, num):
        if len(list(elm.previous_siblings)) == 0:
            return num
        else:
            return Extender.contents_position_loop(elm.previous_sibling, num + 1)
    
    @staticmethod
    def has_tom(tag):
        return 'tom' in tag.attrs

    @staticmethod
    def has_nathan(tag):
        return 'nathan' in tag.attrs

    @staticmethod
    def check_siblings(sibs, attFunc):
        for sib in sibs:
            if attFunc(sib):
                return True
        return False
    
    ## Missle Functions ------------------------------------
    ##
    def mother_to_tom_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.tom is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.tom.parent, motherElement, start).contents[Extender.contents_position(self.vp.tom)]

    def mother_to_nathan_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.nathan is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.nathan.parent, motherElement, start).contents[Extender.contents_position(self.vp.nathan)]

    @staticmethod
    def cycle_up_mother(elm, motherElement, start):
        if elm is motherElement:
            return start
        else:
            return Extender.cycle_up_mother(elm.parent, motherElement, start).contents[Extender.contents_position(elm)]
        
    
    ## Rocket Functions ------------------------------------
    ##
    def nathan_to_tom_rocket(self):
        ## Compute the route from nathan startnode to tom by starting at tom and recursing up, accross and then down
        ## looing for nathan = 0 attribute
        return lambda start: Extender.cycle_up(self.vp.tom, 'nathan', start)

    def tom_to_nathan_rocket(self):
        ## Compute the route from tom startnode to nathan by starting at nathan and recursing up, accross and then down
        ## looing for tom = 0 attibute
        return lambda start: Extender.cycle_up(self.vp.nathan, 'tom', start)
    
    @staticmethod
    def cycle_up(elm, to, start):
        ## Set Attribut Search Function
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## First look Left for shoulder in siblings
        if Extender.check_siblings(elm.previous_siblings, attrFunc):
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling
        ## Then look right
        elif Extender.check_siblings(elm.next_siblings, attrFunc):
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling
        ## if nothing here go up
        else:
            return Extender.cycle_up(elm.parent, to, start).contents[Extender.contents_position(elm)]

    @staticmethod
    def cycle_left(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling

    @staticmethod
    def cycle_right(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling

    @staticmethod
    def cycle_down(elm, to, start):
        return Extender.cycle_down_loop(elm[to], start)

    @staticmethod
    def cycle_down_loop(dist, start):
        ## Start Node Condition
        if dist == 0:
            return start
        else: 
            return Extender.cycle_down_loop(dist - 1, start).parent
 
        

In [243]:
class RocketOnlyExtender(Extender):
    def __init__(self, gm, pointer):
        Extender.__init__(self, gm, pointer)

In [13]:
## Identify Grandmother elements
pointers = vh.verifiedPointers
gmElements = []
gmMatrix = []

for i in range(len(pointers)):
    igmElements = []
    for j in range(i):
        ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
        if pointers[i].get_mother_element() is pointers[j].get_mother_element():
            gm = pointers[i].get_mother_element()
        else:
            gm = pointers[i].common_parent(pointers[j])
        # Append Match to Grand Mother Matrix
        igmElements.append(gm)
        
        # Check to see if this is a new grand mother element,
        # if so append to the gmElements list of unique grandmother elements 
        if gm not in gmElements:
            gmElements.append(gm)
            
    # Append Matrix Row
    gmMatrix.append(igmElements)
    
grandMotherMatrix = np.matrix(gmMatrix)

In [14]:
noGm = len(gmElements)

In [15]:
noGm

1

In [21]:
## if 1 grandmother element test set it and test it
gm = gmElements[0]

In [29]:
def distinct_gm(gm, pts):
    if len(pts) == 0:
        return True
    if gm is pts[0].get_mother_element():
        return False
    else:
        return distinct_gm(gm, pts[1:])

In [31]:
## check to see if Grandmother is Distinct
distinct = distinct_gm(gm, pointers)

In [32]:
distinct

True

In [25]:
pointers[1].get_mother_element()

<div class="team-member-right"><h3 class="team-member-name">S. Monique Wells</h3><h4 class="team-member-position">Director of Finance and Administration</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">Monique Wells is the Director of Finance and Administration for the National Association for Multi-ethnicity in Communications’ (NAMIC). Prior to that she was their Manager of Operations, a position she assumed in August, 2009. Monique is no stranger to NAMIC and has been a member since 1991. She served as a consultant to NAMIC for a number of special projects, prior to joining the staff. As Director of Finance and Administration, she oversees the administrative and financial operations of the national office. An accomplished professional with a diverse background in sales and project management, Monique previously served as an Energy Conservation Analyst with the Los Angeles Unified School District where she oversaw the utility management for sch

In [122]:
## Tag tree with pattern from nathan to tom
def tag_nathans(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.nathan.parent, 'nathan', 0)

def tag_toms(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.tom.parent, 'tom', 0)

def parent_cycle_up(motherElement, element, atr, num):
    ## for the bertha case we stop at the element that is right befor the mother elemenet
    if element.parent is motherElement:
        element[atr] = num
        element['sib'] = atr
    else:
        element[atr] = num
        return(motherElement, element.parent, atr, num + 1)

def reset_tree(pt):
    ## Clear all tom, nathan and sib atributes
    gm = pt.get_mother_element()
    
    for tomTag in gm.find_all(has_tom):
        del tomTag['tom']    
    for nathanTag in gm.find_all(has_nathan):
        del nathanTag['nathan']    

In [35]:
## For Each pointer
# Tag Toms and Nathans
vp = pointers[0]
tag_nathans(vp)
tag_toms(vp)

In [40]:
vp.tom.parent

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [41]:
## Rocket Extraction: Are you ready to launch?
vp.nathan.parent['class']

['team-member-name']

In [42]:
vp.tom.parent['class']

['team-member-position']

In [44]:
vp.get_mother_element()['class']

['team-member-right']

In [46]:
## Tom Starts
tomStarts = gm.find_all(class_=vp.tom.parent['class'])

In [48]:
## Nathan Starts
nathanStarts = gm.find_all(class_=vp.nathan.parent['class'])

In [52]:
## Mother Starts
motherStarts = gm.find_all(class_=vp.get_mother_element()['class'])

In [58]:
vp.tom.parent.previous_sibling

<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [54]:
vp.tom.parent.previous_sibling

'Susan Waldman'

In [55]:
vp.nathan.parent.next_sibling

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [62]:
# Getting to a nathan with a tom start
n = 5
tomStarts[n].previous_sibling.string

'Darryl Jackson'

In [65]:
# Getting to a tom with a nathan Start
n = 0
nathanStarts[n].next_sibling.string

'President and CEO'

In [114]:
tom_rocket = lambda startNode: startNode.next_sibling.string


In [115]:
n = 7
tom_rocket(nathanStarts[n])

'Administrative Assistant'

In [116]:
nathan_rocket = lambda startNode: startNode.previous_sibling.string

In [117]:
n = 7
nathan_rocket(tomStarts[n])

"Davana O'Brien"

In [118]:
nathan_missile = lambda startNode: startNode.contents[0].string

In [119]:
n = 7
nathan_missile(motherStarts[n])

"Davana O'Brien"

In [120]:
tom_missile = lambda startNode: startNode.contents[0].next_sibling.string

In [121]:
n = 7
tom_missile(motherStarts[n])

'Administrative Assistant'

In [ ]:
## begin recursions
## We are tagged and ready

def nathan_to_tom_rocket(pt):
    if 
    
def tom_to_nathan_rocket(pt):
    
def mother_to_tom_missile(pt):
    
def mother_to_nathan_missile(pt):
    


In [123]:
vp.get_mother_element()

<div class="team-member-right"><h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3><h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Researc

In [ ]:
#chain demo 1
def next_sib_gen(n):
    return lambda n:  

In [129]:
next_sib_gen = lambda elm, n: elm if n == 0 else next_sib_gen(elm.next_sibling, n - 1)

In [134]:
next_sib_gen(vp.get_mother_element().contents[0],2)

<div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Research Manager at Vibe, Spin and The Source magazines and Research Director at Motor Trend magazine. Supporting 20 sales representatives by providing them with competitive and analytic research regarding ea

In [143]:
## USELESS, but good attempt.

next_sib = lambda elm: elm.next_sibling
prev_sib = lambda elm: elm.previous_sibling
parent_up = lambda elm: elm.parent
child_down = lambda elm: elm.contents[0]
string_out = lambda elm: elm.string

In [144]:
string_out(prev_sib(tomStarts[0]))

'Eglon E. Simons'

In [146]:
string_out(prev_sib)

AttributeError: 'function' object has no attribute 'string'

In [148]:
rocketString = ".previous_sibling.string"

In [150]:
## We have a winner! The rocketString eval model
nathan_rocket_lambda = lambda elm: eval('elm' + rocketString)

In [ ]:
## This one has promise, look at it!
def left_right (arg):
    if arg == 'Left'
        return lambda elm: left_right(right)

In [166]:
## This one has promise, look at it! Pure higher order functions no strings
two_right = lambda elm: tratorx(1, elm).string

In [167]:
def tratorx(n, elm):
    if n == 0:
        return elm
    else:
        return tratorx(n-1,elm).next_sibling

In [168]:
## Another Winner!
two_right(nathanStarts[0])
## Time to fly!!

'President and CEO'

## Missile Work

In [177]:
## Helpers
def contents_position(elm):
    return contents_position_loop(elm, 0)

def contents_position_loop(elm, num):
    if len(list(elm.previous_siblings)) == 0:
        return num
    else:
        return contents_position_loop(elm.previous_sibling, num + 1)
        
def has_tom(tag):
    return 'tom' in tag.attrs

def has_nathan(tag):
    return 'nathan' in tag.attrs

def check_siblings(sibs, attFunc):
    for sib in sibs:
        if attFunc(sib):
            return True
    return False
        

In [191]:
def mother_to_tom_missile(pt):
    motherElement = pt.get_mother_element()
    
    if pt.tom is motherElement:
        return lambda start: start
    else:
        return lambda start: cycle_up_mother(pt.tom.parent, motherElement, start).contents[contents_position(pt.tom)]
    
def mother_to_nathan_missile(pt):
    motherElement = pt.get_mother_element()
    
    if pt.nathan is motherElement:
        return lambda start: start
    else:
        return lambda start: cycle_up_mother(pt.nathan.parent, motherElement, start).contents[contents_position(pt.nathan)]

def cycle_up_mother(elm, motherElement, start):
    if elm is motherElement:
        return start
    else:
        return cycle_up_mother(elm.parent, motherElement, start).contents[contents_position(elm)]
    

In [179]:
tom_missile = mother_to_tom_missile(vp)

In [180]:
tom_missile

<function __main__.mother_to_tom_missile.<locals>.<lambda>>

In [190]:
n = 7
tom_missile(motherStarts[n])

'Administrative Assistant'

In [192]:
nathan_missile = mother_to_nathan_missile(vp)

In [201]:
n = 6
nathan_missile(motherStarts[n])

'Brandon D. Hewlett'

## Rocket Time !!

In [207]:
def nathan_to_tom_rocket(pt):
    ## Compute the route from nathan startnode to tom by starting at tom and recursing up, accross and then down
    ## looing for nathan = 0 attribute
    return lambda start: cycle_up(pt.tom, 'nathan', start)
    
def tom_to_nathan_rocket(pt):
    ## Compute the route from tom startnode to nathan by starting at nathan and recursing up, accross and then down
    ## looing for tom = 0 attibute
    return lambda start: cycle_up(pt.nathan, 'tom', start)
    
def cycle_up(elm, to, start):
    ## Set Attribut Search Function
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## First look Left for shoulder in siblings
    if check_siblings(elm.previous_siblings, attrFunc):
        return cycle_left(elm.previous_sibling, to, start).next_sibling
    ## Then look right
    elif check_siblings(elm.next_siblings, attrFunc):
        return cycle_right(elm.next_sibling, to, start).previous_sibling
    ## if nothing here go up
    else:
        return cycle_up(elm.parent, to, start).contents[contents_position(elm)]
    
def cycle_left(elm, to, start):
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## if this is the shoulder change functions but dont move
    if attrFunc(elm):
        return cycle_down(elm, to, start)
    ## if not check the element to the left
    else:
        return cycle_left(elm.previous_sibling, to, start).next_sibling
    
def cycle_right(elm, to, start):
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## if this is the shoulder change functions but dont move
    if attrFunc(elm):
        return cycle_down(elm, to, start)
    ## if not check the element to the left
    else:
        return cycle_right(elm.next_sibling, to, start).previous_sibling
    
def cycle_down(elm, to, start):
    return cycle_down_loop(elm[to], start)

def cycle_down_loop(dist, start):
    ## Start Node Condition
    if dist == 0:
        return start
    else: 
        return cycle_down_loop(dist - 1, start).parent
    

In [208]:
## Rocket Tests
tom_rocket = nathan_to_tom_rocket(vp)

In [217]:
n = 7

tom_rocket(nathanStarts[n])

'Administrative Assistant'

In [219]:
nathan_rocket = tom_to_nathan_rocket(vp)

In [227]:
## Success
n = 7
nathan_rocket(tomStarts[n])

"Davana O'Brien"

## Vehicle Assembley
#### nathan_to_tom_rocket
#### tom_to_nathan_rocket
#### mother_to_nathan_missile
#### mother_to_tom_missile

#### Are all recursive constructors, and are used to get to targets by adding to the 'rocketString' variable recuscursively. Once the base case is met, a single lambda is returned which combines the element and the string of opporations in the rocket string.

### Here we go, finnish strong!

In [153]:


nathan_rocket_lambda(tomStarts[0])

'Eglon E. Simons'

In [96]:
motherStarts[0].contents[0].string

'Eglon E. Simons'

In [36]:
vp.nathan.parent


<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [19]:
vp.tom.parent

<h4 class="team-member-position">Senior Manager, Membership and Publications</h4>

In [20]:
vp.mary

<div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Research Manager at Vibe, Spin and The Source mag